In [6]:
import sys, os
sys.path.insert(0, "/home/storage/hans/jax_reco_new")
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from collections import defaultdict

import jax.numpy as jnp
from jax.scipy import optimize
import jax
jax.config.update("jax_enable_x64", True)
import optimistix as optx

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from lib.simdata_i3 import I3SimBatchHandlerTFRecord
from lib.simdata_i3 import I3SimHandler

from lib.experimental_methods import get_clean_pulses_fn_v
from lib.network import get_network_eval_v_fn
from lib.geo import cherenkov_cylinder_coordinates_w_rho_v as cherenkov_cylinder_coordinates_w_rho_v
from lib.geo import get_xyz_from_zenith_azimuth
from lib.cgamma import c_multi_gamma_prob, c_multi_gamma_sf
from lib.plotting import adjust_plot_1d

from dom_track_eval import get_eval_network_doms_and_track2 as get_eval_network_doms_and_track

import time

from collections import defaultdict

In [7]:
dtype = jnp.float32
eval_network_v = get_network_eval_v_fn(bpath='/home/storage/hans/jax_reco_new/data/network',
                                       dtype=dtype)
eval_network_doms_and_track = get_eval_network_doms_and_track(eval_network_v, dtype=dtype)

In [8]:
event_id = '11232'

bp = '/home/storage2/hans/i3files/alerts/bfrv2/'
sim_handler = I3SimHandler(os.path.join(bp, f'meta_ds_event_{event_id}_N100_from_0_to_100_1st_pulse_charge_correction.ftr'),
                              os.path.join(bp, f'pulses_ds_event_{event_id}_N100_from_0_to_100_1st_pulse_charge_correction.ftr'),
                              '/home/storage/hans/jax_reco/data/icecube/detector_geometry.csv')

In [9]:
sigma = 0.7

meta, pulses = sim_handler.get_event_data(0)
print(meta)
# Get dom locations, first hit times, and total charges (for each dom).
event_data = sim_handler.get_per_dom_summary_from_sim_data(meta, pulses)
event_data_corr = sim_handler.get_per_dom_summary_from_sim_data(meta, pulses, charge_key = f'corrected_charge_{sigma:.1f}')

print(event_data)

event_id                   0.000000e+00
idx_start                  0.000000e+00
idx_end                    1.193000e+03
n_channel_HLC              1.160000e+02
neutrino_energy            1.454914e+06
muon_energy                9.485573e+05
muon_energy_at_detector             NaN
muon_energy_lost                    NaN
q_tot                      2.566175e+03
n_channel                  1.270000e+02
muon_zenith                2.388777e+00
muon_azimuth               3.014376e+00
muon_time                  2.728580e+03
muon_pos_x                -1.852532e+03
muon_pos_y                -1.913848e+02
muon_pos_z                -1.304696e+03
spline_mpe_zenith          2.377873e+00
spline_mpe_azimuth         3.045150e+00
spline_mpe_time            1.031543e+04
spline_mpe_pos_x          -3.099220e+02
spline_mpe_pos_y          -3.863327e+02
spline_mpe_pos_z           3.560553e+02
Name: 0, dtype: float64
     sensor_id  charge       x       y       z     time
0            0   1.025 -256.14 -521.08  

In [5]:
print(event_data_corr['charge'].max())

140.48990222946546


In [6]:
hit_x = []
hit_y = []
hit_z = []
hit_t = []
hit_q = []
hit_q_corr = []

for i in range(100):
    meta, pulses = sim_handler.get_event_data(i)
    #print(f"muon energy: {meta['muon_energy_at_detector']/1.e3:.1f} TeV")

    track_time = meta['muon_time']
    track_pos = [meta['muon_pos_x'], meta['muon_pos_y'], meta['muon_pos_z']]
    track_dir = [meta['muon_zenith'], meta['muon_azimuth']]
    
    # Get dom locations, first hit times, and total charges (for each dom).
    event_data = sim_handler.get_per_dom_summary_from_sim_data(meta, pulses)
    event_data_corr = sim_handler.get_per_dom_summary_from_sim_data(meta, pulses, 
                                                                    charge_key = f'corrected_charge_{sigma:.1f}')
    
    hit_t.append(event_data['time'].values - track_time)
    hit_x.append(event_data['x'].values)
    hit_y.append(event_data['y'].values)
    hit_z.append(event_data['z'].values)
    hit_q.append(event_data['charge'].values)
    hit_q_corr.append(event_data_corr['charge'].values)

In [7]:
dom_data = defaultdict(lambda: defaultdict(list))

for i in range(len(hit_x)):
    for j in range(len(hit_x[i])):
        q = hit_q[i][j]
        q_corr = hit_q_corr[i][j]
    
        x = hit_x[i][j]
        y = hit_y[i][j]
        z = hit_z[i][j]
    
        t = hit_t[i][j]
        dom_data[(x,y,z)]['first_hit_time'].append(t)
        dom_data[(x,y,z)]['q_tot'].append(q)
        dom_data[(x,y,z)]['q_tot_corr'].append(q_corr)

for key in dom_data.keys():
    qs = dom_data[key]['q_tot']
    dom_data[key]['mean_q_tot'] = np.median(qs)

    qs = dom_data[key]['q_tot_corr']
    dom_data[key]['mean_q_tot_corr'] = np.median(qs)

In [8]:
#sorting = 'distance'
sorting = 'charge'
pdf = PdfPages(f"test_{event_id}_sigma_{sigma:.1f}_dist_r0.5.pdf")

dom_pos = []
for key in dom_data.keys():
    dom_pos.append(jnp.array(key).reshape((1,3)))

dom_pos = jnp.concatenate(dom_pos, axis=0)
track_pos = jnp.array(track_pos)
track_dir = jnp.array(track_dir)

# evaluate network for these doms
# notice that all true track vertex and directions are the same. So we use the first one.
logits, av, bv, geo_time = eval_network_doms_and_track(dom_pos, track_pos, track_dir)
mix_probs = jax.nn.softmax(logits)

# get also the other DOM info
track_dir_xyz = get_xyz_from_zenith_azimuth(track_dir)

geo_time, closest_approach_dist, closest_approach_z, closest_approach_rho = \
            cherenkov_cylinder_coordinates_w_rho_v(dom_pos,
                                         track_pos,
                                         track_dir_xyz)

# convert first_hit_times to delay_times by subtracting geo_times
# and add more dom data to dom_data dict
for i in range(len(dom_pos)):
    pos = tuple(np.array(dom_pos[i]))
    gt = geo_time[i]
    for j in range(len(dom_data[pos]['first_hit_time'])):
        dom_data[pos]['first_hit_time'][j] -= float(gt)

    dom_data[pos]['closest_approach_dist'] = closest_approach_dist[i]
    dom_data[pos]['closest_approach_z'] = closest_approach_rho[i]
    dom_data[pos]['closest_approach_rho'] = closest_approach_z[i]
    dom_data[pos]['mix_probs'] = np.array(mix_probs[i])
    dom_data[pos]['a'] = np.array(av[i])
    dom_data[pos]['b'] = np.array(bv[i])

# charge sorted mapping
dom_positions = list(dom_data.keys())

if sorting == 'charge':
    dom_positions.sort(key=lambda x: dom_data[x]['mean_q_tot'], reverse=True)
else:
    dom_positions.sort(key=lambda x: dom_data[x]['closest_approach_dist'] ,reverse=False)

n_plots = len(dom_positions)
n_doms_per_page = 3
xvals = np.linspace(-10, 3000, 30000)
sigma = 3.0
delta = 0.01

c_multi_gamma_prob_vx = jax.vmap(c_multi_gamma_prob, (0, None, None, None, None, None), 0)
c_multi_gamma_sf_vx = jax.vmap(c_multi_gamma_sf, (0, None, None, None, None), 0)

#for i in range(0, int(np.min([n_plots, 60])), n_doms_per_page):
for i in range(0, 100, n_doms_per_page):
        print(i)
        fig, ax = plt.subplots(n_doms_per_page, 3)
        for j in range(n_doms_per_page):
            pos = tuple(dom_positions[i])
            g_mix_p = dom_data[pos]['mix_probs']
            g_a = dom_data[pos]['a']
            g_b = dom_data[pos]['b']
            mode = (g_a[1]-1)/g_b[1]

            dist = dom_data[pos]['closest_approach_dist']
            z = dom_data[pos]['closest_approach_z']
            rho = dom_data[pos]['closest_approach_rho']
            for k in range(3):
                tax = ax[j, k]

                if k == 0:
                    tax.set_title(f"event {event_id} (dist={dist:.1f}m, z ={z:.0f}m, rho={rho:.0f}deg)", fontsize=6)
                    yval = c_multi_gamma_prob_vx(xvals, g_mix_p, g_a, g_b, sigma, delta)
                    tax.plot(xvals, yval, label='SPE PDF', color='black')
                    tax.set_ylim([0.0, 1.2 * np.amax(yval)])

                elif k == 1:
                    n_p_orig = dom_data[pos]['mean_q_tot']
                    n_p_orig = np.round(n_p_orig+0.5)
                    n_p = np.floor(np.min([3.6*np.exp(0.23*dist)+1, n_p_orig]))
                    n_p = np.clip(n_p, None, 30)

                    n_p_corr = dom_data[pos]['mean_q_tot_corr']
                    n_p_corr = np.round(n_p_corr+0.5)
                    n_p_corr = np.floor(np.min([3.6*np.exp(0.23*dist)+1, n_p_corr]))
                    
                    probs = c_multi_gamma_prob_vx(xvals, g_mix_p, g_a, g_b, 3.0, 0.1)
                    sfs = c_multi_gamma_sf_vx(xvals, g_mix_p, g_a, g_b, 3.0)

                    yval1 = n_p_orig * probs * jnp.power(sfs, n_p_orig-1)
                    tax.plot(xvals, yval1, label=f'MPE PDF (q={n_p_orig})', color='tab:blue', lw=1)

                    yval2 = n_p * probs * jnp.power(sfs, n_p-1)
                    tax.plot(xvals, yval2, label=f'MPE PDF (q={n_p})', color='tab:orange', lw=1)

                    yval3 = n_p_corr * probs * jnp.power(sfs, n_p_corr-1)
                    tax.plot(xvals, yval3, label=f'MPE PDF (q={n_p_corr})', color='tab:red', lw=2)
                    tax.set_ylim([0.0, 1.2*np.amax([np.amax(yval1), np.amax(yval2), np.amax(yval3)])])


                if k == 0 or k == 1:
                    tax.set_xlabel('delay time [ns]')
                    tax.set_ylabel('pdf')
                    for tx in dom_data[pos]['first_hit_time']:
                        if tx > -20:
                            tax.axvline(tx, alpha=0.1, color='black', lw=0.5)

                    idx = np.array(dom_data[pos]['first_hit_time']) > -20
                    xmax = np.max([20,  1.2 * np.amax(dom_data[pos]['first_hit_time'])])
                    tax.hist(np.array(dom_data[pos]['first_hit_time'])[idx], density=True, alpha=0.5,
                                 label='first hit (from MC)', color='tab:green')
                    tax.legend(fontsize=5)
                    tax.set_xlim([-20 , np.max([20, 3 * mode])])

                if k == 2:
                    tax.hist(dom_data[pos]['q_tot'], color='tab:blue', histtype='step', lw=2)
                    tax.hist(dom_data[pos]['q_tot'], color='tab:blue', alpha=0.5)
                    tax.axvline(dom_data[pos]['mean_q_tot'], alpha=0.5, color='black', lw=1)
                    tax.set_xlabel('total charge [p.e.]')
                    tax.set_ylabel('counts')


            i+=1

        plt.tight_layout(pad=0.2, w_pad=0.2, h_pad=1.0)

        pdf.savefig(fig)
        plt.close()

pdf.close()

0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84


/home/hans/.local/share/virtualenvs/py3_jax_latest-mr9UFGRS/lib/python3.11/site-packages/numpy/lib/histograms.py:885: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


87
90
93
96
99
